# IMPACT EXTRACTOR:
**Description:** The following code will pull impacts from the GLANSIS impact data tables. Furthermore, it will format the impact descriptions into bullet points with in-text citations for GLANSIS Organism Impact Assessments and create a reference section. This code will export an excel file with impacts and a word document with references. 


## Import libraries


In [1]:
import pandas as pd              # Manage data tables
import requests                  # Pulls HTML code from webpage
from bs4 import BeautifulSoup    # HTML parsing
import re                        # Edit text strings
from docx import Document        # Create and edit Word Document
from tkinter import filedialog   # Creates file dialog box
import os                        # Opens documents automatically
from selenium import webdriver                                     # automate web browser interaction
from selenium.webdriver.chrome.options import Options              # use to select 'headless' browser options
from selenium.webdriver.common.by import By                        # find elements by HTML id on webpage
from selenium.webdriver.support.ui import Select                   # use to automate dropdown selection
from selenium.webdriver.support.ui import WebDriverWait            # command driver to wait until web page loaded
from selenium.webdriver.support import expected_conditions as EC   # wait until web page condition met
import openpyxl

## Enter Species ID
Entering an impact ID is optional. The code will grab any impacts IDs after the entered ID.

In [2]:
species_id = ''
first_impact_id = ''


## Table Extraction and Exportation: 
Running the following code will pull the impact data and reformat it for export.

In [20]:
# DO NOT EDIT!

# set URL
url = 'https://nas.er.usgs.gov/queries/greatlakes/Impacts/ImpactsInfo.aspx?speciesID=' + species_id

# Open web page using a headless Selenium webdriver
chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options = chrome_options)
driver.get(url)

# Find dropdown elements on the web page and select information - TYPE
type_dropdown = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'body_ResultsPerPageDD'))))
type_dropdown.select_by_visible_text(str(300))

# Get html script from web page and find RefNum
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

# Select the table
table = soup.find('table', {"id": "body_myGridView"})

# Close the webdriver
driver.quit()

# Create blank list to hold table data
data = []

# Extract table information
for row in table.find_all('tr')[2:-2]:

    cells = row.find_all(['th', 'td'])
    
    row_data = [cell.get_text(strip = True, separator = " ") for cell in cells]

    data.append(row_data)

# Convert list to DataFrame
impact_table = pd.DataFrame(data[1:], columns = data[0])

# Filter rows based on condition
if first_impact_id != 'NA':
    
    # Find row index of first impact id numbe
    selected_rows = impact_table[impact_table['Impact ID'] >= first_impact_id]
    
else:
    
    # Duplicate impact_table
    selected_rows = impact_table.copy()


# Create a copy of the DataFrame
selected_rows = selected_rows.copy()

# Create empty column
selected_rows['NAS_Reference'] = None

# Pull cut-and-paste references from NAS
for index, row in selected_rows.iterrows():
    
    # Redo with Selenium
    url = 'https://nas.er.usgs.gov/queries/references/ReferenceViewer.aspx?refnum=' + str(row['Reference'])
    
    # Call url
    response = requests.post(url)
     
    # Scrape the RefNum 
    if response.status_code == 200:
        
        # Get HTML
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find specific tag using id attribute
        desired_span_tag = soup.find("span", {"id": "ContentPlaceHolder1_CutPasteRef"})
        
        # Fill 'NAS_Reference' column
        if desired_span_tag:
            
            # Pull cut-and-paste reference from HTML
            reference = desired_span_tag.get_text(strip = True)
            
        else:
            
            # If error, set reference to error
            reference = 'Error'
            
        # Fill cell
        selected_rows.loc[index, 'NAS_Reference'] = reference
            
# Formula to format citations
def format_citation(citation):
    
    # Extract year
    year = re.findall(r'\d{4}', citation)[0]
    
    # Extract everything before the period before the year using regex
    match = re.match(r'^(.+?)\.\s\d{4}\.', citation).group(1).strip()
    
    # Split names portion of reference
    ref_parts = match.split(', ')
    
    # Count name reference parts
    n_authors = len(ref_parts) - 1
    
    # Create in-text citation for impact descriptions
    if n_authors == 1:
        
        # Combinbe first author name with year
        in_text_citation = str(ref_parts[0]) + ' ' + str(year)

    elif n_authors == 2:

        # Check if 'and' appears in the second text in the list
        if 'and' in ref_parts[2]:

            # Split the second text by space and select the last word
            second_author = ref_parts[2].split(' ')[-1].strip()
            
            # Combine two author names together with year
            in_text_citation = str(ref_parts[0]) + ' and ' + str(second_author) + ' ' + str(year)

        else:
            # In case of comma splice in one-author references - this combines author last name with year
            in_text_citation = str(ref_parts[0]) + ' ' + str(year)

    else:
        
        # Combine lead author with 'et al.' and year
        in_text_citation = str(ref_parts[0]) + ' et al. ' + str(year)
        
    return(in_text_citation)


# Create column with in-text citations
selected_rows['in_text'] = selected_rows['NAS_Reference'].apply(format_citation)


# Function to combine text and additional information, removing the last period from the text
def combine_text(row):
    
    text = row['Impact Description']
    
    # Remove the last character (period)
    if row['Impact Description'].endswith('.'):
        row['Impact Description'] = row['Impact Description'][:-1]  
        
    combined = str(row['Impact Description']) + ' ' + '(' + str(row['in_text']) + ').'
        
    return combined


# Apply the function to each row to create a new column
selected_rows['Impact Description'] = selected_rows.apply(combine_text, axis=1)

# Reneame columns
new_column_names = {'Reference': 'RefNum', 'NAS_Reference': 'Reference'}
selected_rows.rename(columns = new_column_names, inplace = True)

# Select necessary columns 
new_column_order = ["Impact ID", "Impact Type", "Study Type", "Study Location", "Impact Description", "Geographic Location", "RefNum", "Reference"]
selected_rows = selected_rows[new_column_order]


## Create Excel file with revised impact descriptions

In [21]:
# Set file path
excel_file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")])

# Save Excel file
selected_rows.to_excel(excel_file_path, engine='openpyxl', index=False)


## Create Word Document with alphabetically ordered references

In [4]:
# Extract and sort references
references = sorted(selected_rows['Reference'])

# Create a new Word document
doc = Document()

# Add a title to the document
doc.add_heading('References', level = 1)

# Add references to the document
for reference in references:
    doc.add_paragraph(reference)

# Set file path
root = Tk()                                         
root.attributes("-topmost", True)                   
root.withdraw()
doc_file_path = filedialog.asksaveasfilename(defaultextension = ".docx", filetypes = [("Word Document", "*.docx")])

# Save the document
doc.save(doc_file_path)
